In [1]:
# 모듈 로딩
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib as plt
import seaborn as sns
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

c:\Users\KDP-14\anaconda3\envs\commate\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# 데이터 로딩
DATA_PATH = './KNN 전처리(2.9~3.1).csv'
dataDF = pd.read_csv(DATA_PATH)

### train_test 나누기

In [3]:
dataDF

,c_temp_pv,k_rpm_pv,n_temp_pv,scale_pv,s_temp_pv
0,69.6,189.0,67.2,3.01,67.1
1,69.8,189.0,67.2,3.01,67.0
2,69.7,189.0,67.9,3.08,65.9
3,69.7,189.0,67.8,3.08,65.9
4,69.7,189.0,67.8,3.08,65.9
...,...,...,...,...,...
139779,70.4,185.0,66.7,3.06,66.9
139780,70.4,185.0,66.6,3.03,66.8
139781,70.3,185.0,66.7,3.03,66.7
139782,70.4,185.0,66.7,3.04,66.6


In [4]:
target = dataDF['scale_pv']
feature = dataDF.drop(columns='scale_pv')

In [5]:
target

0         3.01
1         3.01
2         3.08
3         3.08
4         3.08
          ... 
139779    3.06
139780    3.03
139781    3.03
139782    3.04
139783    3.05
Name: scale_pv, Length: 139784, dtype: float64

In [6]:
feature

,c_temp_pv,k_rpm_pv,n_temp_pv,s_temp_pv
0,69.6,189.0,67.2,67.1
1,69.8,189.0,67.2,67.0
2,69.7,189.0,67.9,65.9
3,69.7,189.0,67.8,65.9
4,69.7,189.0,67.8,65.9
...,...,...,...,...
139779,70.4,185.0,66.7,66.9
139780,70.4,185.0,66.6,66.8
139781,70.3,185.0,66.7,66.7
139782,70.4,185.0,66.7,66.6


In [7]:
X_train, X_test, y_train, y_test = train_test_split(feature,target, test_size= 0.2, random_state=77)

### feature 데이터 스케일링
- Standard, MinMax, Robust

In [8]:
# from sklearn.preprocessing import StandardScaler
# # standard scaler 선언 및 학습
# standardScaler = StandardScaler().fit(X_train)

# # train셋 내 feature들에 대하여 standard scaling 수행
# X_train_standard = standardScaler.transform(X_train)

# # test셋 내 feature들에 대하여 standard scaling 수행
# X_test_standard = standardScaler.transform(X_test)

In [9]:
# 학석들이 발견한 좋은 조합
xgb = XGBRegressor()
parameters = {
    'max_depth': [3, 7],
    'n_estimators': [500, 1000],
    'learning_rate': [0.03, 0.05, 0.07, 0.1],
    'colsample_bytree': [0.5, 0.7],
    'subsample': [0.5, 0.7, 0.8],
    'min_child_weight': [1, 5],
    'random_sate':[2],
    'n_thread': [4], 
}

xgb_grid = GridSearchCV(xgb, parameters, cv=5, n_jobs=5, verbose=5)
xgb_grid.fit(X_train.values, y_train)
scores = pd.DataFrame(xgb_grid.cv_results_)
scores[['params','mean_test_score', 'rank_test_score', 'split0_test_score','split1_test_score', 'split2_test_score']]

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)

Fitting 5 folds for each of 192 candidates, totalling 960 fits
[11:19:01] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_thread", "random_sate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


0.4141807101951084
{'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 1000, 'n_thread': 4, 'random_sate': 2, 'subsample': 0.8}


In [10]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
# xgbBest = xgb_grid.best_estimator_
# scores = pd.DataFrame(xgb_grid.cv_results_)
# pd.set_option('display.max_seq_items', None)
# pd.set_option('display.max_columns', 21)
# scores.info()

# print(scores.sort_values(by=scores.columns[20], ascending=True).head(10))

In [11]:
dataDF.iloc[64]

c_temp_pv     69.50
k_rpm_pv     189.00
n_temp_pv     69.30
scale_pv       3.03
s_temp_pv     71.00
Name: 64, dtype: float64

In [12]:
X_train_np = X_train.values

In [13]:
## 가장 좋은 파라미터 값
## {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'min_child_weight': 5, 'n_estimators': 1000, 'n_thread': 4, 'random_sate': 2, 'subsample': 0.8}

myxgb = XGBRegressor(colsample_bytree=0.5, learning_rate=0.1, max_depth=7, min_child_weight=5, n_estimators=1000, n_thread= 4, random_state=2, subsample=0.8)
myxgb.fit(X_train_np, y_train)


[11:19:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:576: 
Parameters: { "n_thread" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.5, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_delta_step=0,
             max_depth=7, min_child_weight=5, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=8, n_thread=4,
             num_parallel_tree=1, predictor='auto', random_state=2, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [14]:
y_pred = myxgb.predict(X_test)
MAE = mean_absolute_error(y_test,y_pred)
MAPE = mean_absolute_percentage_error(y_test,y_pred)
Score = myxgb.score(X_test,y_test)
print(f'MAE => {MAE}     MAPE => {MAPE*100}    R2 => {Score}')

MAE => 0.01138583316087305     MAPE => 0.3747927379798579    R2 => 0.4074683319699619


In [15]:
myxgb.feature_importances_

array([0.1686993 , 0.4537321 , 0.21148486, 0.16608383], dtype=float32)

In [16]:
y_pred1=myxgb.predict(np.array([70.2, 189, 69.8, 70.5]).reshape(1,-1))
print(y_pred1)

[3.0134225]
